In [1]:
# RT PIC - classic and nearest neighbour
import underworld as uw
import math
from underworld import function as fn
import glucifer.pylab as plt

In [2]:
dim = 2

In [3]:
# create mesh objects
elementMesh = uw.mesh.FeMesh_Cartesian( elementType='Q1/dQ0', 
                                         elementRes=(64,64), 
                                           minCoord=(0.,0.), 
                                           maxCoord=(0.9142,1.) )
linearMesh   = elementMesh
constantMesh = elementMesh.subMesh 

In [4]:
# create fevariables
velocityField = uw.fevariable.FeVariable( feMesh=linearMesh,   nodeDofCount=dim )
pressureField = uw.fevariable.FeVariable( feMesh=constantMesh, nodeDofCount=1 )

In [5]:
# Initialise data.. Note that we are also setting boundary conditions here
velocityField.data[:] = [0.,0.]
pressureField.data[:] = 0.

In [6]:
# Get the actual sets 
#
#  HJJJJJJH
#  I      I
#  I      I
#  I      I
#  HJJJJJJH
#  
#  Note that H = I & J 

# Note that we use operator overloading to combine sets
IWalls = linearMesh.specialSets["MinI_VertexSet"] + linearMesh.specialSets["MaxI_VertexSet"]
JWalls = linearMesh.specialSets["MinJ_VertexSet"] + linearMesh.specialSets["MaxJ_VertexSet"]

In [7]:
# Now setup the dirichlet boundary condition
# Note that through this object, we are flagging to the system 
# that these nodes are to be considered as boundary conditions. 
# Also note that we provide a tuple of sets.. One for the Vx, one for Vy.
AllWalls = IWalls + JWalls

freeslipBC = uw.conditions.DirichletCondition(     variable=velocityField, 
                                              nodeIndexSets=(AllWalls,JWalls) )

In [8]:
# We create swarms of particles which can advect, and which may determine 'materials'
gSwarm = uw.swarm.Swarm( feMesh=elementMesh )

# Now we add a data variable which will store an index to determine material
materialVariable = gSwarm.add_variable( dataType="char", count=1 )

# Layouts are used to populate the swarm across the whole domain
# Create the layout object
layout = uw.swarm.layouts.GlobalSpaceFillerLayout( swarm=gSwarm, particlesPerCell=20 )
# Now use it to populate.
gSwarm.populate_using_layout( layout=layout )

In [9]:
# Lets initialise the 'materialVariable' data to represent two different materials. 
materialHeavyIndex = 0
materialLightIndex = 1

# Now let's initialize the materialVariable with the required perturbation
import math
wavelength = 1.8284
amplitude  = 0.02
offset     = 0.2
k = 2.*math.pi / wavelength
coordinate = fn.input()
materialVariable.data[:] = fn.branching.conditional( 
    [ ( offset + amplitude*fn.math.cos( k*coordinate[0] ) > coordinate[1] , materialLightIndex ),
      (                                                              True , materialHeavyIndex )  ] ).evaluate(gSwarm)

In [10]:
# visualise
fig1 = plt.Figure()
fig1.Points( swarm=gSwarm, colourVariable=materialVariable )
#Test loading a script
sc = fig1.script('rotate y -45')
fig1.show()

In [11]:
#Open interactive viewer
fig1.open_viewer()

In [12]:
#Run a command, get image
fig1.send_command('scale points 1')
fig1.image()
#fig1.close_viewer()

In [13]:
#Close
fig1.close_viewer()

In [14]:
# We create some functions here.
# The Map function allows as to create 'per material' type behaviour.  
# It requires a 'keyFunc', which is first evaluate to determine the key
# into the map, and then once the key is determine, we retrieve the value (ie function)
# it maps to, and evaluate that. 

# Here we set a viscosity value of '1.' for both materials 
viscosityMapFn = fn.branching.map( keyFunc = materialVariable, 
                         mappingDict = { materialLightIndex:1., materialHeavyIndex:1. } )
# Here we set a density of '0.' for the lightMaterial, and '1.' for the heavymaterial.
densityFn      = fn.branching.map( keyFunc = materialVariable,
                         mappingDict = { materialLightIndex:0., materialHeavyIndex:1. } )
# Define our gravity using a python tuple (this will be automatically converted to a function)
gravity = ( 0.0, -1.0 )
# now create a buoyancy force vector.. the gravity tuple is converted to a function 
# here via operator overloading
buoyancyFn = gravity*densityFn

In [15]:
# Setup a stokes system
# For PIC style integration, we include a swarm for the a PIC integration swarm is generated within.
# For gauss integration, simple do not include the swarm. Nearest neighbour is used where required.
stokesPIC = uw.systems.Stokes(velocityField=velocityField, 
                              pressureField=pressureField,
                              swarm=gSwarm, 
                              conditions=[freeslipBC,],
                              viscosityFn=viscosityMapFn, 
                              bodyForceFn=buoyancyFn )

In [16]:
# Create advector objects to advect the swarms. We specify second order integration.
advector = uw.systems.SwarmAdvector( swarm=gSwarm, velocityField=velocityField, order=2 )
# Also create some integral objects which are used to calculate statistics.
v2sum_integral  = uw.utils.Integral( feMesh=linearMesh, fn=fn.math.dot(velocityField,velocityField) )
volume_integral = uw.utils.Integral( feMesh=linearMesh, fn=1. )

In [19]:
# Stepping. Initialise time and timestep.
time = 0.
step = 0
# Perform 10 steps
while step<30:
    # Get solution for initial configuration
    stokesPIC.solve()
    # Retrieve the maximum possible timestep for the advection system.
    dt = advector.get_max_dt()
    # Advect using this timestep size
    advector.integrate(dt)
    # Calculate the RMS velocity
    v2sum = v2sum_integral.integrate()
    volume = volume_integral.integrate()
    vrms = math.sqrt(v2sum[0]/volume[0])
    print 'step =',step, 'time =', time, 'vrms = ', vrms
    # Increment
    time += dt
    step += 1

In [20]:
fig1.show()